In [ ]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES), and is copyright (c) 2018-2022
# by the software owners: The Regents of the University of California, through
# Lawrence Berkeley National Laboratory,  National Technology & Engineering
# Solutions of Sandia, LLC, Carnegie Mellon University, West Virginia University
# Research Corporation, et al.  All rights reserved.
#
# Please see the files COPYRIGHT.md and LICENSE.md for full copyright and
# license information.
###############################################################################

Example from:

Wilson, Zachary T., and Nikolaos V. Sahinidis. "Automated learning of chemical reaction networks." Computers & Chemical Engineering 127 (2019): 88-98.
https://doi.org/10.1016/j.compchemeng.2019.05.020

Case 2: Dynamic Chemical Looping Combusion Reactor

This is an example of a CLC reactor. The kinetic reaction rates encapsulate solid-gas reactions. The kinetic rate laws for this example are semi-physical or empirical to provide insights on the underlying physical mechanisms.

The rate laws are often expressed in terms similar to

$ \frac{dX}{dT} = kA(X)g(F) $

where $ A(X) $ is a mechanism-dependent activity term, and the function $ g(F) $ is a parametric function of processss conditions, typically in this case the partial pressure of methane used as a fuel.   


In [ ]:
# Imports and data

from idaes.apps import ripe
import numpy as np
from idaes.apps.ripe import mechs as mechs


np.random.seed(20)

# Import data from csv
data = np.genfromtxt("clc.csv", delimiter=",")
t = data[:, 0]
xdata = data[:, 1]

# Stoichiometry
# One species
stoich = [1]

We are going to use empirical pre-defined functions from RIPE, defined in the idaes.surrogate.ripe.mechs. The mechanisms depend on only one species for these pre-defined rate forms.

Rate Equation                           | $ A(x) $
:---                                    |                  :--- 
Random Nucleation                       | $ 1-x $ 
Power law $n = 2/3, 1.5, 2, 3, 4 $      | $ nx^{(n-1/n)} $
Avrami-Erofeev $ n = 0.5, 1.5, 2, 3, 4$ | $ n(1-x)(-log(1-x))^{(n-1/n)} $
Prout Tompkins                          | $ x(x-1) $
Jander                                  | $ 3(1-x)^{1/3} (1/(1+x)^{((-1/3)-1)}) $
Antijander                              | $ 3/2(1-x)^{(2/3)}(1/(1+x)^{((-1/3)-1)}) $
Valensi                                 | $ 1/(-log(1-x)) $
Parabolic                               | $ 1/2x $
Ginstling-Brountstein diffusion-3d      | $ (3/2)(1-x)^{(4/3)}/((1-x)^{(-1/3)}-1) $
Zhuralev-Leshokin-Tempelman             | $ (3/2)/((1-x)^{(-1/3)}-1) $
Grain model                             | $ (1-x)^{(2/3)} $





In [ ]:
# User pre-defined clc rate forms found in RIPE
# mechs = ripe.clcforms
clc_mechs = [
    mechs.randomnuc,
    mechs.powerlawp5,
    mechs.powerlaw2,
    mechs.powerlaw3,
    mechs.powerlaw4,
    mechs.avrami2,
    mechs.avrami3,
    mechs.avrami4,
    mechs.avrami5,
    mechs.ptompkins,
    mechs.jander,
    mechs.antijander,
    mechs.valensi,
    mechs.parabolic,
    mechs.gb3d,
    mechs.zlt,
    mechs.grain,
]

All that is left is to run the ripe modeler:

In [ ]:
# Identify optimal kinetic mechanism
results = ripe.ripemodel(xdata, stoichiometry=stoich, mechanisms=clc_mechs, time=t)